## Goal

The goal of this notebook is to demonstrate how to retrieve data from a local Solana cluster.
We will retrieve simple hellos from multiple data accounts, and also show how to listen for updates on a given number of accounts.

In [ ]:
#from python_client.accounts import BaseAccount
from solana.publickey import PublicKey
from solana.rpc.async_api import AsyncClient
from anchorpy import Program, Provider, Wallet
from solana.keypair import Keypair
from base64 import b64decode
import base58
import yaml
from dataclasses import dataclass
import json
import pathlib
import borsh_construct as borsh #from borsh_construct import U8, String, CStruct
import asyncio
from solana.rpc.websocket_api import connect
from asyncstdlib import enumerate

In [2]:
#!pip install pyyaml
#!pip install asyncstdlib

We define a structure for deserializing Solana messages. See ```src/program-rust/src/lib.rs``` and see that this matches the format there.

In [56]:
counter_structure = borsh.CStruct(
     "counter" / borsh.U32,
 )

## Sync data retrieval

Now we want to fetch all initialized accounts that have received hellos. We first fetch our program_id and the derived greeted account.

In [83]:
def get_greeted_pub_key(GREETER_SEED='hello'):
    # programId
    keypair_path = pathlib.Path.cwd().joinpath('dist','program','helloworld-keypair.json')
    with open(keypair_path) as f:
        keypair = Keypair.from_secret_key(bytes(json.load(f)[:32]))

    programId = keypair.public_key

    # payer
    config_path = pathlib.Path.home().joinpath('.config','solana','cli','config.yml')
    config = yaml.safe_load(open(config_path))
    with open(config['keypair_path']) as f:
        payer = Keypair.from_secret_key(bytes(json.load(f)[:32]))
    
    greetedPubkey = PublicKey.create_with_seed(payer.public_key,GREETER_SEED,programId)
    return greetedPubkey, programId

In [84]:
greetedPubkey, programId = get_greeted_pub_key()
greetedPubkey, programId

(2bt1NMJds5g4W1LwAeoVsMcUSa9qPD2PywAapBuLnpZw,
 GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj)

Now we fetch all accounts owned by the program_id.

In [108]:
program_accounts = await client.get_program_accounts(PublicKey(programId))
program_accounts

{'jsonrpc': '2.0',
 'result': [{'account': {'data': 'cahmH',
    'executable': False,
    'lamports': 918720,
    'owner': 'GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj',
    'rentEpoch': 0},
   'pubkey': '2bt1NMJds5g4W1LwAeoVsMcUSa9qPD2PywAapBuLnpZw'},
  {'account': {'data': 'QioWX',
    'executable': False,
    'lamports': 918720,
    'owner': 'GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj',
    'rentEpoch': 0},
   'pubkey': '97j9kXnLJsGcz6Nxu96rN5S6AEqhrkmuYqsWH1nmW632'}],
 'id': 3}

In [115]:
#counter_structure.parse(base58.b58decode('cahmH'))

Now we retrieve the account information for each account.
Note that the ```program_accounts``` object already displays the data, but the base is not available.

In [116]:
async def get_account_info(program_address):
    async with AsyncClient("http://localhost:8899") as client:
        res = await client.is_connected()
        resp= await client.get_account_info(program_address)
        info = resp["result"]["value"]
        data, base = info["data"]
        if base == 'base64':
            bytes_data = b64decode(data)
        elif base == 'base58':
            bytes_data = base58.b58decode(data)
        else:
            raise Exception('base cannot be processed')
        
        print (counter_structure.parse(bytes_data))

In [117]:
for pubKeystr in [i['pubkey'] for i in program_accounts['result']]:
    await get_account_info(PublicKey(pubKeystr))

Container: 
    counter = 24
Container: 
    counter = 16


## Async data retrieval

The Solana RPC also offers a websocket client. We can use that to subscribe to transactions, logs and of course accounts, which is what we will do next.

In [124]:
async def subscribe_accounts(account_pubkey, data_container):
    """
    We subscribe to accounts and output the messages to the list given as input.
    We break after we received 1 message.
    """
    async with connect("ws://localhost:8900") as websocket:
        await websocket.account_subscribe(account_pubkey, encoding='base64')
        first_resp = await websocket.recv()
        subscription_id = first_resp.result
        async for idx, msg in enumerate(websocket):
            
            data_container.append(msg)
            counter, _base = msg.result.value.data
            bytes_data = b64decode(counter)
            print (f'counter - {animal.parse(bytes_data)}')
            
            print (f'msg {msg}')
            
            if idx == 0:
                break
        await websocket.account_unsubscribe(subscription_id)

We trigger the function and are now waiting for msgs to arrive (with infinite timeout). Go ahead and send a hello using ```npm start``` and check back the output here.

In [123]:
msgs = []
await asyncio.gather(
      subscribe_accounts_and_print_enum(greetedPubkey, msgs),
      subscribe_accounts_and_print_enum(anotherGreetedPubKey, msgs)
    )

first_resp Ok(result=67, id=15)
subscription_id 67
first_resp Ok(result=68, id=16)
subscription_id 68
counter - Container: 
    counter = 27
msg AccountNotification(subscription=67, result=AccountInfoAndContext(context=Context(slot=83455), value=AccountInfo(lamports=918720, owner=GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj, data=('GwAAAA==', 'base64'), executable=False, rent_epoch=0)))
counter - Container: 
    counter = 19
msg AccountNotification(subscription=68, result=AccountInfoAndContext(context=Context(slot=83456), value=AccountInfo(lamports=918720, owner=GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj, data=('EwAAAA==', 'base64'), executable=False, rent_epoch=0)))
counter - Container: 
    counter = 28
msg AccountNotification(subscription=67, result=AccountInfoAndContext(context=Context(slot=83517), value=AccountInfo(lamports=918720, owner=GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj, data=('HAAAAA==', 'base64'), executable=False, rent_epoch=0)))
counter - Container: 
    counter 

[None, None]

In [125]:
msgs

[AccountNotification(subscription=67, result=AccountInfoAndContext(context=Context(slot=83455), value=AccountInfo(lamports=918720, owner=GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj, data=('GwAAAA==', 'base64'), executable=False, rent_epoch=0))),
 AccountNotification(subscription=68, result=AccountInfoAndContext(context=Context(slot=83456), value=AccountInfo(lamports=918720, owner=GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj, data=('EwAAAA==', 'base64'), executable=False, rent_epoch=0))),
 AccountNotification(subscription=67, result=AccountInfoAndContext(context=Context(slot=83517), value=AccountInfo(lamports=918720, owner=GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj, data=('HAAAAA==', 'base64'), executable=False, rent_epoch=0))),
 AccountNotification(subscription=68, result=AccountInfoAndContext(context=Context(slot=83518), value=AccountInfo(lamports=918720, owner=GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj, data=('FAAAAA==', 'base64'), executable=False, rent_epoch=0)))]

Now that you have the msgs object, you can process them meaningfully, for example store in a PostgreSQL database.
A plugin (https://github.com/solana-labs/solana-accountsdb-plugin-postgres) was developed for storing transactions long-term in an external PostgreSQL database.